In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent.parent))

In [9]:
import time
import triton
import triton.language as tl
import torch
import torch.nn.functional as F
import pandas as pd
from conv_gemm.triton_kernels.fp16.col2img_kernel import col2img_kernel

In [10]:
def col2img_fp32(
    cols_f32: torch.Tensor,
    N: int, Cin: int,
    H: int, W: int,
    Kh: int, Kw: int,
    Sh: int, Sw: int,
    Ph: int, Pw: int,
    Dh: int, Dw: int,
    BLOCK_M: int,
    BLOCK_K: int,
    num_warps: int = 4,
    num_stages: int = 2,
):
    assert cols_f32.is_cuda
    assert cols_f32.dtype == torch.float32
    cols_f32 = cols_f32.contiguous()

    Ho = (H + 2*Ph - Dh*(Kh-1) - 1)//Sh + 1
    Wo = (W + 2*Pw - Dw*(Kw-1) - 1)//Sw + 1
    M  = N * Ho * Wo
    K  = Cin * Kh * Kw

    assert cols_f32.shape == (M, K), f"cols shape {cols_f32.shape}, expected {(M, K)}"

    x_f32 = torch.zeros((N, Cin, H, W), device=cols_f32.device, dtype=torch.float32)
    sN, sC, sH, sW = x_f32.stride()

    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(K, BLOCK_K))

    col2img_kernel[grid](
        cols_f32, x_f32,
        N, Cin, H, W,
        Kh, Kw, Sh, Sw, Ph, Pw, Dh, Dw,
        Ho, Wo,
        sN, sC, sH, sW,
        K,
        BLOCK_M=BLOCK_M,
        BLOCK_K=BLOCK_K,
        num_warps=num_warps,
        num_stages=num_stages,
    )

    return x_f32

In [14]:
@torch.no_grad()
def bench_once_col2img_fp16_vs_torch(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
    iters=100,
    device="cuda",
):
    """
    FP32 Triton col2img vs torch F.fold    
    """
    Ho = (H + 2*Ph - Dh*(Kh-1) - 1)//Sh + 1
    Wo = (W + 2*Pw - Dw*(Kw-1) - 1)//Sw + 1
    M  = N * Ho * Wo
    K  = Cin * Kh * Kw

    cols_f32 = torch.randn((M, K), device=device, dtype=torch.float32) #для производной fp32

    # Torch F.fold
    cols_fold = cols_f32.view(N, Ho*Wo, K).permute(0, 2, 1).contiguous()

    def _call_torch():
        return F.fold(
            cols_fold,
            output_size=(H, W),
            kernel_size=(Kh, Kw),
            dilation=(Dh, Dw),
            padding=(Ph, Pw),
            stride=(Sh, Sw),
        )

    for _ in range(5):
        _call_torch()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        x_ref = _call_torch()
    torch.cuda.synchronize()
    t_torch = (time.perf_counter() - t0) / iters

    # Triton col2img
    def _call_triton():
        return col2img_fp32(
            cols_f32,
            N, Cin, H, W,
            Kh, Kw,
            Sh, Sw,
            Ph, Pw,
            Dh, Dw,
            BLOCK_M=BLOCK_M,
            BLOCK_K=BLOCK_K,
            num_warps=num_warps,
            num_stages=num_stages,
        )

    # warmup
    for _ in range(5):
        _call_triton()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        x_i32 = _call_triton()
    torch.cuda.synchronize()
    t_triton = (time.perf_counter() - t0) / iters

    # bandwidth
    bytes_moved = (cols_f32.numel() + x_i32.numel()) * 4.0
    bw_triton = bytes_moved / t_triton / 1e9
    bw_torch  = bytes_moved / t_torch  / 1e9

    return {
        "M": M, "K": K, "N": N,
        "BLOCK_M": BLOCK_M,
        "BLOCK_N": 0,
        "BLOCK_K": BLOCK_K,
        "num_warps": num_warps,
        "num_stages": num_stages,
        "t_triton_ms": t_triton * 1e3,
        "t_torch_ms": t_torch * 1e3,
        "speed_vs_torch": t_torch / t_triton,
        "bw_triton_GBs": bw_triton,
        "bw_torch_GBs": bw_torch,
    }

In [15]:
@torch.no_grad()
def tune_col2img_fp16_tiles_for_shape(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(1, 2, 4, 8),
    stages=(2, 3, 4),
    iters=200,
    device="cuda",
):
    records = []
    for BM in blocks_M:
        for BK in blocks_K:
            for Wp in warps:
                for S in stages:
                    try:
                        rec = bench_once_col2img_fp32_vs_torch(
                            N, Cin, H, W,
                            Kh, Kw,
                            Sh, Sw,
                            Ph, Pw,
                            Dh, Dw,
                            BLOCK_M=BM,
                            BLOCK_K=BK,
                            num_warps=Wp,
                            num_stages=S,
                            iters=iters,
                            device=device,
                        )
                    except RuntimeError as e:
                        print(f"[SKIP] BM={BM}, BK={BK}, W={Wp}, S={S}: {e}")
                        continue

                    print(
                        f"BM={BM}, BK={BK}, W={Wp}, S={S}: "
                        f"t_triton={rec['t_triton_ms']:.3f} ms, "
                        f"speed_vs_torch={rec['speed_vs_torch']:.3f}x, "
                    )
                    records.append(rec)

    if not records:
        raise RuntimeError("No valid tile configs found for this COL2IMG shape")

    df = pd.DataFrame(records)
    return df


In [16]:
df_c2i = tune_col2img_fp32_tiles_for_shape(
    N=16, Cin=1, H=256, W=256,
    Kh=11, Kw=11,
    Sh=1, Sw=1,
    Ph=5, Pw=5,
    Dh=1, Dw=1,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(2, 4, 8),
    stages=(2, 3),
    iters=200,
    device="cuda",
)

BM=32, BK=32, W=2, S=2: t_triton=1.934 ms, speed_vs_torch=1.192x, 
BM=32, BK=32, W=2, S=3: t_triton=1.934 ms, speed_vs_torch=1.173x, 
BM=32, BK=32, W=4, S=2: t_triton=1.551 ms, speed_vs_torch=1.413x, 
BM=32, BK=32, W=4, S=3: t_triton=1.555 ms, speed_vs_torch=1.402x, 
BM=32, BK=32, W=8, S=2: t_triton=1.653 ms, speed_vs_torch=1.317x, 
BM=32, BK=32, W=8, S=3: t_triton=1.658 ms, speed_vs_torch=1.324x, 
BM=32, BK=64, W=2, S=2: t_triton=4.482 ms, speed_vs_torch=0.490x, 
BM=32, BK=64, W=2, S=3: t_triton=4.480 ms, speed_vs_torch=0.489x, 
BM=32, BK=64, W=4, S=2: t_triton=1.852 ms, speed_vs_torch=1.191x, 
BM=32, BK=64, W=4, S=3: t_triton=1.883 ms, speed_vs_torch=1.198x, 
BM=32, BK=64, W=8, S=2: t_triton=1.595 ms, speed_vs_torch=1.394x, 
BM=32, BK=64, W=8, S=3: t_triton=1.597 ms, speed_vs_torch=1.376x, 
BM=32, BK=128, W=2, S=2: t_triton=5.865 ms, speed_vs_torch=0.373x, 
BM=32, BK=128, W=2, S=3: t_triton=5.872 ms, speed_vs_torch=0.375x, 
BM=32, BK=128, W=4, S=2: t_triton=4.895 ms, speed_vs_torch=0

In [17]:
df_c2i.sort_values("t_triton_ms").head(10)

,M,K,N,BLOCK_M,BLOCK_N,BLOCK_K,num_warps,num_stages,t_triton_ms,t_torch_ms,speed_vs_torch,bw_triton_GBs,bw_torch_GBs
2,1048576,121,16,32,0,32,4,2,1.551120,2.191647,1.412945,329.893908,233.479724
3,1048576,121,16,32,0,32,4,3,1.555483,2.180833,1.402029,328.968540,234.637464
22,1048576,121,16,64,0,32,8,2,1.593044,2.189388,1.374342,321.212074,233.720576
10,1048576,121,16,32,0,64,8,2,1.595419,2.224389,1.394235,320.733976,230.042992
11,1048576,121,16,32,0,64,8,3,1.596694,2.196895,1.375903,320.477946,232.921939
23,1048576,121,16,64,0,32,8,3,1.600355,2.200891,1.375252,319.744653,232.499018
4,1048576,121,16,32,0,32,8,2,1.653148,2.177344,1.317090,309.533761,235.013421
5,1048576,121,16,32,0,32,8,3,1.657952,2.195820,1.324417,308.636787,233.035960
8,1048576,121,16,32,0,64,4,2,1.851805,2.206045,1.191294,276.327741,231.955921
9,1048576,121,16,32,0,64,4,3,1.882986,2.256372,1.198295,271.751986,226.782198


In [18]:
FP32_COL2IMG_BEST_BLOCK_M = 32
FP32_COL2IMG_BEST_BLOCK_N = 0
FP32_COL2IMG_BEST_BLOCK_K = 32
FP32_COL2IMG_BEST_WARPS = 4
FP32_COL2IMG_BEST_STAGES = 2